In [1]:
# imports
import requests
import pandas as pd
from notion_client import Client
import datetime
import json
import time

In [ ]:
def send_communication(notion_raw_id): 
    
    
   # info needed from Notion : Email, Phone, first name, last name  
   ZOHO_CLIENT_ID = '1000.VO682Z1FM15RTS0EPC3QG9OR3ZA81J' 
   ZOHO_CLIENT_SECRET = '73966cb9ddadac75baa08767dc1e8fd4caaa758b54'
   code = "1000.b4f2262dcb01a3020de73d65ca23f889.0da1dcff24375744507be8b85e388a2b"
    
    
    

In [25]:
def get_zoho_auth_code():
    import webbrowser
    from http.server import HTTPServer, BaseHTTPRequestHandler
    from urllib.parse import urlparse, parse_qs

    scope = 'ZohoMail.messages.ALL,ZohoMail.accounts.READ,ZohoMail.messages.CREATE'
    ZOHO_CLIENT_ID = '1000.VO682Z1FM15RTS0EPC3QG9OR3ZA81J'
    redirect_uri = 'http://localhost&prompt=consent' 
    
    auth_url = f"https://accounts.zoho.com/oauth/v2/auth?scope={scope}&client_id={ZOHO_CLIENT_ID}&response_type=code&access_type=offline&redirect_uri={redirect_uri}"

    class RequestHandler(BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.end_headers()
            self.server.path = self.path

    def run_server():
        server = HTTPServer(('localhost', 80), RequestHandler)
        server.handle_request()
        return server.path

    webbrowser.open(auth_url)
    callback_url = run_server()
    
    parsed_url = urlparse(callback_url)
    auth_code = parse_qs(parsed_url.query)['code'][0]
    
    return auth_code

code = get_zoho_auth_code()

127.0.0.1 - - [12/Feb/2025 11:58:45] "GET /?code=1000.ba9a97f37485d89502283905b2a547a1.521cb4f9ced0e99d1a8a6c363d781251&location=us&accounts-server=https%3A%2F%2Faccounts.zoho.com& HTTP/1.1" 200 -


In [26]:
code

'1000.ba9a97f37485d89502283905b2a547a1.521cb4f9ced0e99d1a8a6c363d781251'

In [27]:
# get aouth token
import requests

def get_zoho_tokens(auth_code, client_id, client_secret):
    """
    Exchange authorization code for access token and refresh token
    """
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    
    data = {
        "code": auth_code,
        "client_id": client_id,
        "client_secret": client_secret,
        "redirect_uri": "http://localhost",
        "grant_type": "authorization_code"
    }
    
    response = requests.post(token_url, data=data)
    return response.json()

# Replace these with your actual values
CLIENT_ID = "1000.VO682Z1FM15RTS0EPC3QG9OR3ZA81J"  # Your client ID
CLIENT_SECRET = "73966cb9ddadac75baa08767dc1e8fd4caaa758b54"  # Your client secret
AUTH_CODE = code  # The code you just received

# Get the tokens
result = get_zoho_tokens(AUTH_CODE, CLIENT_ID, CLIENT_SECRET)

# Print the results
if "access_token" in result:
    print("\nAccess Token:", result["access_token"])
    print("\nRefresh Token:", result["refresh_token"])
    print("\nExpires in:", result["expires_in"], "seconds")
else:
    print("Error:", result)



Access Token: 1000.8577832de9cbef0e3aa925947399bf51.1e10c39da860079682ede58f5a104865

Refresh Token: 1000.8e189778917bde78b6b8fd1fced5a6f8.b34bcab0a7248cf421fc8608c134348e

Expires in: 3600 seconds


In [1]:
import requests

def refresh_access_token(refresh_token, client_id, client_secret):
    """
    Get a new access token using the refresh token
    """
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    
    data = {
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "refresh_token"
    }
    
    response = requests.post(token_url, data=data)
    return response.json()

# Example usage
if __name__ == "__main__":
    # Your credentials
    CLIENT_ID = "1000.VO682Z1FM15RTS0EPC3QG9OR3ZA81J"  # Your client ID
    CLIENT_SECRET = "73966cb9ddadac75baa08767dc1e8fd4caaa758b54"  # Your client secret
    REFRESH_TOKEN = "1000.8e189778917bde78b6b8fd1fced5a6f8.b34bcab0a7248cf421fc8608c134348e"  # This is the long-lived token you received initially
    
    # Get new access token
    result = refresh_access_token(REFRESH_TOKEN, CLIENT_ID, CLIENT_SECRET)
    
    if "access_token" in result:
        print("\nNew Access Token:", result["access_token"])
        print("\nExpires in:", result["expires_in"], "seconds")
    else:
        print("Error:", result)



New Access Token: 1000.421844717b4a48a74686fd4e04fe0356.21734a46e5936e5efcc4181f9067c800

Expires in: 3600 seconds


In [6]:
result['access_token']

'1000.421844717b4a48a74686fd4e04fe0356.21734a46e5936e5efcc4181f9067c800'

In [8]:
import requests

def get_zoho_account_id(auth_token):
    """Get Zoho Mail account ID using the access token"""
    url = "https://mail.zoho.com/api/accounts"
    
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }
        
    try:
        response = requests.get(url, headers=headers)
        print(f"Response Status: {response.status_code}")
        print(f"Response Body: {response.text}")
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"Error: {response.text}")
            return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Your access token from the previous step
AUTH_TOKEN = "1000.421844717b4a48a74686fd4e04fe0356.21734a46e5936e5efcc4181f9067c800"

# Get account information
account_info = get_zoho_account_id(AUTH_TOKEN)


Response Status: 200
Response Body: {"status":{"code":200,"description":"success"},"data":[{"country":"il","lastLogin":1739346052570,"mxStatus":true,"activeSyncEnabled":false,"mobileNumber":"","incomingBlocked":false,"language":"iw","type":"ZOHO_ACCOUNT","extraStorage":{},"incomingUserName":"info@trueclaim.org","emailAddress":[{"isAlias":false,"isPrimary":true,"mailId":"info@trueclaim.org","isConfirmed":true}],"mailboxStatus":"enabled","popBlocked":false,"usedStorage":0,"spamcheckEnabled":true,"imapAccessEnabled":false,"timeZone":"Asia/Jerusalem","accountCreationTime":1739012699591,"zuid":878306757,"webBlocked":false,"planStorage":5,"firstName":"info","accountId":"3454657000000008002","sequence":1,"mailboxAddress":"info@trueclaim.org","lastPasswordReset":1739346044581,"tfaEnabled":false,"iamStatus":1,"phoneNumer":"","status":true,"lastName":"info","accountDisplayName":"info","role":"member","gender":"NOT_SPECIFIED","accountName":"trueclaim","displayName":"info info","isLogoExist":false

In [4]:
import requests
import json

def send_email_zoho(from_email, to_email, subject, content, auth_token):
    """
    Send an email using ZOHO Mail API
    
    Parameters:
    - from_email: Sender's email address
    - to_email: Recipient's email address
    - subject: Email subject
    - content: Email content
    - auth_token: OAuth access token
    """
    
    # API endpoint with your account ID
    url = "https://mail.zoho.com/api/accounts/3454657000000008002/messages"
    
    # Headers
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }
    
    # Request body
    payload = {
        "fromAddress": from_email,
        "toAddress": to_email,
        "subject": subject,
        "content": content,
        "mailFormat": "html"
    }
    
    try:
        # Send POST request
        response = requests.post(url, headers=headers, json=payload)
        
        # Check if request was successful
        if response.status_code == 200:
            print("Email sent successfully!")
            return response.json()
        else:
            print(f"Failed to send email. Status code: {response.status_code}")
            print(f"Response: {response.text}")
            return response.json()
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    # Your authenticated credentials
    AUTH_TOKEN = "1000.7b76f88749d11381e67b2f58a8669f92.a077a6bcb86ea66f840f2c8421f96c85"
    FROM_EMAIL = "info@trueclaim.org"  # Your Zoho email
    TO_EMAIL = "info@trueclaim.org"  # Replace with the recipient's email
    SUBJECT = "Test Email from ZOHO API"
    CONTENT = """
    <html>
        <body>
            <h1>Hello!</h1>
            <p>This is a test email sent using the ZOHO Mail API.</p>
        </body>
    </html>
    """
    
    # Send the email
    result = send_email_zoho(
        FROM_EMAIL,
        TO_EMAIL,
        SUBJECT,
        CONTENT,
        AUTH_TOKEN
    )


Failed to send email. Status code: 404
Response: {"data":{"errorCode":"INVALID_OAUTHTOKEN"},"status":{"code":404,"description":"Invalid Input"}}


In [7]:
result['data']['errorCode'] == 'INVALID_OAUTHTOKEN'

True